In [1]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import pickle

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

In [114]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [2]:
def scroll_bottom():
    counter = 0

    while True:
        counter += 1 
        
        if counter >= 500:
            break
    
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(10)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(10)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue

In [116]:
def get_wine_url(base_url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660DQ1WSwYSLmoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-OT80rwStXzblNTiZLXykuhYoFIwZQSlTCC0MYQyh1AmasW2yYkA-pkm0g%3D%3D&cart_item_source=nav-explore'):
    driver.get(base_url)
    
    #scroll
    scroll_bottom()
    
    #get links
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #wine_page_links = [link.get('href').strip() for link in soup.find_all('a', class_='anchor__anchor--2QZvA')]
    wine_page_links = []
    for link in soup.find_all('a', class_='anchor__anchor--2QZvA'):
        if "year=" in link.get('href').strip() and "cart_item_source" in link.get('href').strip():
            wine_page_links.append(link.get('href').strip())        
        else:
            pass
    
    return wine_page_links

In [117]:
wine_page_links = get_wine_url()

In [118]:
len(wine_page_links)

4574

In [1]:
type(soup)

NameError: name 'soup' is not defined

In [119]:
wine_page_links[:10]


['/veuve-clicquot-brut-carte-jaune-champagne/w/1128385?year=N.V.&cart_item_source=direct-explore',
 '/moet-chandon-imperial-brut-champagne/w/1122095?year=N.V.&cart_item_source=direct-explore',
 '/casal-garcia-vinho-verde-branco/w/1226459?year=N.V.&cart_item_source=direct-explore',
 '/farnese-edizione-cinque-autoctoni/w/21646?year=N.V.&cart_item_source=direct-explore',
 '/piccini-memoro-rosso-tuscany/w/1155726?year=N.V.&cart_item_source=direct-explore',
 '/louis-roederer-brut-premier-champagne/w/74298?year=N.V.&cart_item_source=direct-explore',
 '/bollinger-special-cuvee-brut-ay-champagne/w/18929?year=N.V.&cart_item_source=direct-explore',
 '/taittinger-brut-reserve-champagne/w/1129619?year=N.V.&cart_item_source=direct-explore',
 '/mateus-the-original-rose/w/1253734?year=N.V.&cart_item_source=direct-explore',
 '/perrier-jouet-grand-brut-champagne/w/79160?year=N.V.&cart_item_source=direct-explore']

In [12]:
def parse_ratings(soup):
    rating_raw = soup.find_all("div", class_=lambda value: value and value.startswith("vivinoRatingWide"))[0]
    rating_clean = rating_raw.text.split()[0]
    return rating_clean


In [13]:
def parse_winery(soup):
    winery_raw = soup.find(class_="winery")
    winery_clean = winery_raw.text.strip('\n')
    return winery_clean

In [14]:
def parse_wine_name(soup):
    name_raw = soup.find(class_="vintage")
    name_raw2 = name_raw.text.split()[:-1]
    name_clean = ' '.join(name_raw2)
    return name_clean

In [15]:
def parse_wine_vintage(soup):
    vintage_raw = soup.find(class_="vintage")
    vintage_clean = vintage_raw.text.split()[-1]
    return vintage_clean

In [16]:
def parse_wine_type(soup):
    type_raw = soup.find(class_="wineLocationHeader__wineType--14nrC")
    type_clean = type_raw.text
    return type_clean

In [17]:
def parse_wine_region(soup):
    region_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__region--1cbip")
    region_clean = region_raw.text
    return region_clean

In [18]:
def parse_wine_country(soup):
    country_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__country--1RcW2")
    country_clean = country_raw.text
    return country_clean

In [19]:
def parse_number_reviews(soup):   
    reviews_raw = soup.find(class_="vivinoRatingWide__basedOn--s6y0t")
    reviews_clean = reviews_raw.text[:-8]
    return reviews_clean

In [71]:
def parse_avg_price(soup):
    price = soup.find("span", class_=lambda value: value and value.startswith("purchaseAvailability"))
    return price.text[1:]
    

In [107]:
def scrape_wine_info():
    avg_rating_list = []
    winery_list = []
    name_list = []
    vintage_list =[]
    type_list = []
    region_list = []
    country_list = []
    num_reviews_list = []
    avg_price_list = []

    base_url = 'https://www.vivino.com'
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    
    for page in wine_page_links:
        time.sleep(10)
        driver.get(base_url + page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')      

        try:
            rating = parse_ratings(soup)
            avg_rating_list.append(rating)
        except:
            avg_rating_list.append('error')
            
        try:
            winery = parse_winery(soup)
            winery_list.append(winery)
        except:
            winery_list.append('error')
        
        try:
            name = parse_wine_name(soup)
            name_list.append(name)
        except:
            name_list.append('error')
            
        try:
            year = parse_wine_vintage(soup)
            vintage_list.append(year)
        except:
            vintage_list.append('error')
            
        try:
            wine_type = parse_wine_type(soup)
            type_list.append(wine_type)
        except:
            type_list.append('error') 
            
        try:
            region = parse_wine_region(soup)
            region_list.append(region)
        except:
            region_list.append('error')   
        
        try:
            country = parse_wine_country(soup)
            country_list.append(country)
        except:
            country_list.append('error')
        
        try:
            num_reviews = parse_number_reviews(soup)
            num_reviews_list.append(num_reviews)
        except:
            num_reviews_list.append('error')
            
        try:
            avg_price = parse_avg_price(soup)
            avg_price_list.append(avg_price)
        except:
            avg_price_list.append('error')
         
            
    features = ('Average Rating', 'Winery', 'Wine Name', 'Vintage',
                'Type', 'Region', 'Country', 'Number of Reviews',
               'Average Price')
    df = pd.DataFrame(list(zip(avg_rating_list, winery_list, name_list, vintage_list, type_list, region_list, 
                               country_list, num_reviews_list, avg_price_list)), columns =features) 
    
    return df

In [120]:
df = scrape_wine_info()

In [121]:
df.head()

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,4.298098,Veuve Clicquot,Brut (Carte Jaune) Champagne,N.V.,Sparkling wine,Champagne,France,98098,49.99
1,4.187012,Moët & Chandon,Impérial Brut Champagne,N.V.,Sparkling wine,Champagne,France,87012,error
2,3.767924,Casal Garcia,Vinho Verde Branco,N.V.,White wine,Vinho Verde,Portugal,67924,error
3,4.352855,Farnese,Edizione Cinque Autoctoni,N.V.,Red wine,Abruzzo,Italy,52855,error
4,3.847411,Piccini,Memoro Rosso,N.V.,Red wine,Toscana,Italy,47411,10.99


In [122]:
#pickle
with open('wine_data_longer.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [123]:
with open('wine_data_longer.pkl', 'rb') as picklefile: 
    wine_data = pickle.load(picklefile)

In [124]:
wine_data

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,4.298098,Veuve Clicquot,Brut (Carte Jaune) Champagne,N.V.,Sparkling wine,Champagne,France,98098,49.99
1,4.187012,Moët & Chandon,Impérial Brut Champagne,N.V.,Sparkling wine,Champagne,France,87012,error
2,3.767924,Casal Garcia,Vinho Verde Branco,N.V.,White wine,Vinho Verde,Portugal,67924,error
3,4.352855,Farnese,Edizione Cinque Autoctoni,N.V.,Red wine,Abruzzo,Italy,52855,error
4,3.847411,Piccini,Memoro Rosso,N.V.,Red wine,Toscana,Italy,47411,10.99
...,...,...,...,...,...,...,...,...,...
4569,3.8(3197),Rodney Strong,Cabernet Sauvignon,2016,Red wine,Sonoma County,United States,,error
4570,3.9(3195),Villa Maria,Private Bin Sauvignon Blanc,2013,White wine,Marlborough,New Zealand,,error
4571,3.6(3195),Quinta do Crasto,Flor de Crasto Tinto,2015,Red wine,Douro,Portugal,,10.76
4572,3.4(3193),Dancing Flame,Ojos del Salado Shiraz,2018,Red wine,Central Valley,Chile,,4.99
